<a href="https://colab.research.google.com/github/hollyisyoon/TIL/blob/main/%ED%9B%88%EC%98%81_%EB%91%90%EC%9E%87.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import math as mt
import numpy as np
import pandas as pd
from scipy.stats import norm
from scipy import stats
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind
from scipy.stats import mannwhitneyu

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Sample Dataset 생성

In [ ]:
import pandas as pd
import numpy as np

# Set random seed for reproducibility
np.random.seed(42)

# Number of samples
n_samples = 2000

# Generate A group data with random variations around AS-IS values
a_data = {
    'Purchase CVR': np.random.normal(0.15, 0.015, n_samples),
    'Team Open CVR': np.random.normal(0.03, 0.003, n_samples),
    'Home→Purchase Lead Time': np.random.normal(120, 12, n_samples),
    'Home→PDP CVR': np.random.normal(0.30, 0.03, n_samples),
    '총 주문 건 수': np.random.normal(4500/7, 450/7, n_samples),  # divided by 7 for daily values
    '인당 주문건 수': np.random.normal(2.42, 0.242, n_samples),
    '팀 성사율(3인 매칭)': np.random.normal(0.30, 0.03, n_samples),
    '평균 배송 시간': np.random.normal(1800, 180, n_samples)
}

# Create dataframe for A group
df_a = pd.DataFrame(a_data)

# Ensure all percentages are within 0 and 1
df_a['Purchase CVR'] = np.clip(df_a['Purchase CVR'], 0, 1)
df_a['Team Open CVR'] = np.clip(df_a['Team Open CVR'], 0, 1)
df_a['Home→PDP CVR'] = np.clip(df_a['Home→PDP CVR'], 0, 1)
df_a['팀 성사율(3인 매칭)'] = np.clip(df_a['팀 성사율(3인 매칭)'], 0, 1)

# Generate B group data with random variations around A group values
variation = np.random.uniform(0.95, 1.10, size=(n_samples, len(a_data.keys())))

# Create dataframe for B group
df_b = df_a * variation

# Ensure all percentages are within 0 and 1
df_b['Purchase CVR'] = np.clip(df_b['Purchase CVR'], 0, 1)
df_b['Team Open CVR'] = np.clip(df_b['Team Open CVR'], 0, 1)
df_b['Home→PDP CVR'] = np.clip(df_b['Home→PDP CVR'], 0, 1)
df_b['팀 성사율(3인 매칭)'] = np.clip(df_b['팀 성사율(3인 매칭)'], 0, 1)

# Generate random timestamps over 7 days for each sample
start_time = pd.Timestamp.now().normalize()  # start from midnight today
end_time = start_time + pd.Timedelta(days=7)

# Generate random timestamps for A and B datasets
timestamps_a = np.random.choice(pd.date_range(start_time, end_time, freq='T'), size=n_samples, replace=False)
timestamps_b = np.random.choice(pd.date_range(start_time, end_time, freq='T'), size=n_samples, replace=False)

# Sorting timestamps for clarity
timestamps_a = np.sort(timestamps_a)
timestamps_b = np.sort(timestamps_b)

# Adding timestamps to the dataframes
df_a['Timestamp'] = timestamps_a
df_b['Timestamp'] = timestamps_b

# Reordering columns to have 'Timestamp' as the first column
df_a = df_a[['Timestamp'] + [col for col in df_a if col != 'Timestamp']]
df_b = df_b[['Timestamp'] + [col for col in df_b if col != 'Timestamp']]

df_a.head(), df_b.head()

# Add 'Group' column to both dataframes
df_a['Group'] = 'A'
df_b['Group'] = 'B'

# Combine both dataframes
combined_df = pd.concat([df_a, df_b], ignore_index=True)
combined_df.head(3)

,Timestamp,Purchase CVR,Team Open CVR,Home→Purchase Lead Time,Home→PDP CVR,총 주문 건 수,인당 주문건 수,팀 성사율(3인 매칭),평균 배송 시간,Group
0,2023-08-05 00:01:00,0.157451,0.027974,109.638077,0.266578,640.734090,2.255804,0.335244,1759.636628,A
1,2023-08-05 00:09:00,0.147926,0.029566,119.625558,0.281072,610.479626,2.346069,0.243631,1762.049751,A
2,2023-08-05 00:12:00,0.159715,0.027623,120.216202,0.271738,631.775896,2.275434,0.290166,2202.376938,A


## Success Metric

In [ ]:
data = pd.read_csv('/content/drive/Shareddrives/서쿼드/2기/Project/훈영/experiment_data.csv')
data.head(3)

,Timestamp,Purchase CVR,Team Open CVR,Home→Purchase Lead Time,Home→PDP CVR,총 주문 건 수,인당 주문건 수,팀 성사율(3인 매칭),평균 배송 시간,Group
0,2023-08-05 00:01:00,0.157451,0.027974,109.638077,0.266578,640.734090,2.255804,0.335244,1759.636628,A
1,2023-08-05 00:09:00,0.147926,0.029566,119.625558,0.281072,610.479626,2.346069,0.243631,1762.049751,A
2,2023-08-05 00:12:00,0.159715,0.027623,120.216202,0.271738,631.775896,2.275434,0.290166,2202.376938,A


In [ ]:
def compute_stats(metric_name, group_A, group_B):
    """Compute mean values, improvement rate, and p-value for a given metric."""

    #평균
    mean_A = group_A[metric_name].mean()
    mean_B = group_B[metric_name].mean()

    #개선율
    improvement_rate = (mean_B - mean_A) / mean_A

    #p-value
    if "CVR" in metric_name or "CTR" in metric_name:
        p_value = ttest_ind(group_A[metric_name], group_B[metric_name]).pvalue
    else:
        p_value = mannwhitneyu(group_A[metric_name], group_B[metric_name]).pvalue

    return mean_A, mean_B, improvement_rate, p_value

metrics_updated = [
    'Purchase CVR',
    'Team Open CVR',
    'Home→Purchase Lead Time',
    'Home→PDP CVR',
    '총 주문 건 수',
    '인당 주문건 수',
    '팀 성사율(3인 매칭)',
    '평균 배송 시간'
]
results_updated = []
for metric in metrics_updated:
    results_updated.append(compute_stats(metric, group_A, group_B))

results_df_updated = pd.DataFrame(results_updated, columns=["Group A Mean", "Group B Mean", "Improvement Rate", "P-value"], index=metrics_updated)
results_df_updated['p-value<0.05'] = results_df_updated['P-value']<0.05
results_df_updated

,Group A Mean,Group B Mean,Improvement Rate,P-value,p-value<0.05
Purchase CVR,0.150676,0.154481,0.025248,2.272280e-14,True
Team Open CVR,0.029981,0.030730,0.024987,1.289927e-13,True
Home→Purchase Lead Time,119.423933,122.389860,0.024835,2.000643e-12,True
Home→PDP CVR,0.299866,0.307233,0.024569,1.240114e-12,True
총 주문 건 수,643.059887,659.679535,0.025845,7.673942e-14,True
인당 주문건 수,2.413817,2.469577,0.023100,4.175507e-11,True
팀 성사율(3인 매칭),0.301090,0.308497,0.024599,7.005854e-13,True
평균 배송 시간,1802.154808,1850.294168,0.026712,5.390432e-14,True
